In [2]:
import util;
import os;
import visualize;
import numpy as np;
import time;

def getNumNeighbors(file_horse):
    lines=util.readLinesFromFile(file_horse);
    lines=np.array(lines);
    uni_lines=np.unique(lines);
    counts=np.zeros(uni_lines.shape);
    for idx_uni_curr,uni_curr in enumerate(uni_lines):
        if idx_uni_curr%500==0:
            print idx_uni_curr;
        counts[idx_uni_curr]=np.sum(lines==uni_curr);
    return uni_lines,counts;

def writeSmallDatasetFile(out_file_pre,horse_data,num_neighbor,
                          num_data,in_file_horse,in_file_face,in_file_face_noIm,post_tags=None):
    if post_tags is None:
        post_tags=['_horse.txt','_face.txt','_face_noIm.txt'];
        
    in_files=[in_file_horse,in_file_face,in_file_face_noIm];
    
    data_org=util.readLinesFromFile(in_file_horse);
    data_org=np.array(data_org);
    idx_keep_all=[];
    print horse_data.shape
    horse_data=horse_data[:num_data];
    for horse_curr in horse_data:
        idx_curr=np.where(data_org==horse_curr)[0];
        idx_curr=np.sort(idx_curr)
        idx_keep=idx_curr[:num_neighbor];
        idx_keep_all=idx_keep_all+list(idx_keep);
#         print num_data,idx_keep
        
    idx_keep_all=np.array(idx_keep_all);
    print idx_keep_all.shape
    files_to_return=[];
    for idx_in_file,in_file in enumerate(in_files):
        out_file_curr=out_file_pre+post_tags[idx_in_file];
        if idx_in_file==0:
            data_keep=data_org[idx_keep_all];
        else:
            data_curr=util.readLinesFromFile(in_file);
            data_curr=np.array(data_curr);
            data_keep=data_curr[idx_keep_all];
        util.writeFile(out_file_curr,data_keep);
        files_to_return.append(out_file_curr);
    
    return files_to_return;
        
def main():
    dir_server='/home/SSD3/maheen-data';
    click_str='http://vision1.idav.ucdavis.edu:1000';
    
    out_dir_meta_data='/home/SSD3/maheen-data/horse_project/data_check';
    dir_neighbors='/home/SSD3/maheen-data/horse_project/neighbor_data';
    matches_file=os.path.join(dir_neighbors,'horse_trainImageList_2_data_100_neigbors.txt');
    
    out_dir_debug=os.path.join(dir_neighbors,'debug');
    out_dir_breakdowns=os.path.join(dir_neighbors,'small_datasets');
    file_pre='matches';
    util.mkdir(out_dir_breakdowns);
    util.mkdir(out_dir_debug);
    
    out_file_counts=os.path.join(out_dir_debug,'counts.npz');
    out_file_dist=os.path.join(out_dir_debug,'counts_dist.png');
    
    out_dir_meta_horse = os.path.join(out_dir_meta_data,'horse');
    out_dir_meta_face = os.path.join(out_dir_meta_data,'aflw');
    
    num_neighbors=100;
    out_file_face=os.path.join(out_dir_meta_face,'matches_'+str(num_neighbors)+'_train_allKP.txt');
    out_file_face_noIm=os.path.join(out_dir_meta_face,'matches_'+str(num_neighbors)+'_train_allKP_noIm.txt');
    out_file_horse=os.path.join(out_dir_meta_horse,'matches_'+str(num_neighbors)+'_train_allKP.txt');
    
    old_horse_file='/home/SSD3/maheen-data/horse_project/data_check/horse/matches_5_train_allKP_minLoss_clean_full.txt';
    old_data=util.readLinesFromFile(old_horse_file);
    old_data=np.array(old_data);
    old_data=np.unique(old_data);

#     new_data,counts=getNumNeighbors(out_file_horse);
#     np.savez(out_file_counts,new_data=new_data,counts=counts);
    
    data=np.load(out_file_counts);
    print data.files;
    new_data=data['new_data'];
    counts=data['counts'];
    num_data=range(500,len(old_data),500);
    num_data[-1]=len(old_data);
    
    num_neighbors=range(5,25,5);
    np.random.shuffle(old_data);
    for num_neighbor in num_neighbors:
        for num_data_curr in num_data:
            file_curr=file_pre+'_'+str(num_neighbor)+'_'+str(num_data_curr);
            out_file_pre=os.path.join(out_dir_breakdowns,file_curr);
            files=writeSmallDatasetFile(out_file_pre,old_data,num_neighbor,num_data_curr,out_file_horse,\
                                        out_file_face,out_file_face_noIm);
            for file_curr in files:
                print file_curr;
    


    
def writeMinLossFile(out_file_pre,post_tags,minloss_post,old_horse_file,old_human_file,old_human_file_noIm):
    new_files=[out_file_pre+post_tag_curr for post_tag_curr in post_tags];
    
    old_data=util.readLinesFromFile(old_horse_file);
    old_data=np.array(old_data);
    
    new_data=util.readLinesFromFile(new_files[0]);
    new_data=np.array(new_data);
    new_data_uni=np.unique(new_data);
    bin_keep=np.in1d(old_data,new_data_uni);
#     print bin_keep.shape,sum(bin_keep);
    old_files=[old_horse_file,old_human_file,old_human_file_noIm];
    new_files_write=[file_curr[:file_curr.rindex('.')]+minloss_post for file_curr in new_files];
    for old_file_curr,new_file_curr,new_file_org in zip(old_files,new_files_write,new_files):
        data_curr=util.readLinesFromFile(old_file_curr);
        data_curr=np.array(data_curr);
        data_keep=data_curr[bin_keep];
        print old_file_curr,new_file_curr,len(data_keep);
        print data_keep[0];
        new_file_org_data=util.readLinesFromFile(new_file_org);
        new_file_org_data=np.array(new_file_org_data);
        bin_check=np.in1d(data_keep,new_file_org_data);
        print sum(bin_check),data_keep.shape[0];
        assert sum(bin_check)==data_keep.shape[0];
#         util.writeFile(new_file_curr,data_keep);
    
def getCommandFaceTest(path_to_th,out_dir,file_curr,batch_size=100):
    command=['th',path_to_th];
    command=command+['-val_data_path',file_curr];
    command=command+['-outDir',out_dir];
    command=command+['-batchSize',str(batch_size)];
    amount_data=len(util.readLinesFromFile(file_curr));
    num_iterations=amount_data/batch_size;
    if amount_data%batch_size!=0:
        num_iterations=num_iterations+1;
    command=command+['-iterations',str(num_iterations)];
    command=' '.join(command);
    return command;

def getCommandFaceTrain(path_to_th,out_dir,file_curr):
    command=['th',path_to_th];
    command=command+['-data_path',file_curr];
    command=command+['-outDir',out_dir];
    command=' '.join(command);
    return command;

def getCommandFull2LossTrain(path_to_th,out_dir,horse_data_path,human_data_path,tps_model_path):
    command=['th',path_to_th];
    command=command+['-outDir',out_dir];
    command=command+['-horse_data_path',horse_data_path];
    command=command+['-human_data_path',human_data_path];
    command=command+['-tps_model_path',tps_model_path];
    command=' '.join(command);
    return command;


def writeMinLossFileLossData(out_file_pre,post_tags,minloss_post,loss_file):
    new_files=[out_file_pre+post_tag_curr for post_tag_curr in post_tags];
    horse_data=util.readLinesFromFile(new_files[0]);
    horse_data=np.array(horse_data);
    horse_data_uni=np.unique(horse_data);
    face_data=util.readLinesFromFile(new_files[1]);
    face_data_noIm=util.readLinesFromFile(new_files[2]);
    assert len(face_data)==len(face_data_noIm);
    
    loss_all=np.load(loss_file);
    loss_all=loss_all[:len(face_data)];
    assert loss_all.shape[0]==len(face_data);
    
    new_data=[[],[],[]];
    for idx_curr,horse_curr in enumerate(horse_data_uni):
        idx_rel=np.where(horse_data==horse_curr)[0];
        loss_rel=loss_all[idx_rel];
        min_idx=np.argmin(loss_rel);
        min_idx_big=idx_rel[min_idx];
        assert loss_rel[min_idx]==loss_all[min_idx_big];
        new_data[0].append(horse_curr);
        new_data[1].append(face_data[min_idx_big]);
        new_data[2].append(face_data_noIm[min_idx_big]);
  
    new_files_out=[new_file_curr[:new_file_curr.rindex('.')]+minloss_post for new_file_curr in new_files];
    for new_file_to_write,data_to_write in zip(new_files_out,new_data):
        print new_file_to_write,len(data_to_write);
        util.writeFile(new_file_to_write,data_to_write);

def getFilePres():
    num_neighbors=range(5,10,5);
    num_data=range(500,3500,500);
    num_data=num_data+[3531]
    post_tags='_horse_minloss';
    file_pre='matches';
    file_pres=[file_pre+'_'+str(num_neighbors[0])+'_'+str(num_data_curr)+post_tags for num_data_curr in num_data];
    return file_pres,num_data

def getLogFileLoss(log_file):
    data_curr=util.readLinesFromFile(log_file);
    data_curr=data_curr[-1];
    data_curr=data_curr.split(' ')[-1].strip('"');
    data_curr=float(data_curr);
    return data_curr;

def getMinLoss(dir_meta,loss_dir_pre,loss_dir_posts,log_file):
    loss_all=[];
    for loss_dir_post in loss_dir_posts:
        file_curr=os.path.join(dir_meta,loss_dir_pre+loss_dir_post,log_file);
        if os.path.exists(file_curr):
            loss=getLogFileLoss(file_curr);
        else:
            loss=float('inf');
        loss_all.append(loss);
        
    loss_all=np.array(loss_all);
    min_idx=np.argmin(loss_all);
    min_loss=loss_all[min_idx];
    return min_loss,loss_dir_posts[min_idx];


dir_server='/home/SSD3/maheen-data';
click_str='http://vision1.idav.ucdavis.edu:1000';
print 'hello'


hello


In [ ]:
def main_4():
    num_neighbors=range(5,25,5);
    num_data=range(500,3500,500);
    num_data=num_data+[3531]
    post_tags=['_horse.txt','_face.txt','_face_noIm.txt'];
    
    dir_neighbors='/home/SSD3/maheen-data/horse_project/neighbor_data';
    out_dir_breakdowns=os.path.join(dir_neighbors,'small_datasets');
    file_pre='matches';
    minloss_post='_minloss.txt';
    commands_all=[];
    files_human=[];
    
    out_dir_min_loss=os.path.join(dir_neighbors,'min_loss_calculation');
    
    for num_neighbor in num_neighbors:
        for num_data_curr in num_data:
            file_curr=file_pre+'_'+str(num_neighbor)+'_'+str(num_data_curr);
            out_file_pre=os.path.join(out_dir_breakdowns,file_curr); 
            loss_dir=os.path.join(out_dir_min_loss,out_file_pre+post_tags[1][:post_tags[1].rindex('.')]);
            loss_file=os.path.join(loss_dir,'test_images','loss_final_val_ind.npy');
            if os.path.exists(loss_file):
                writeMinLossFileLossData(out_file_pre,post_tags,minloss_post,loss_file);
            else:
                pass;


In [ ]:
def tps_script():
    num_neighbors=[5]
#     range(5,25,5);
    num_data=range(1000,3500,500);
    num_data=num_data+[3531]
    post_tags=['_horse.txt','_face.txt','_face_noIm.txt'];
    dir_neighbors='/home/SSD3/maheen-data/horse_project/neighbor_data';
    out_dir_exp='/home/SSD3/maheen-data/horse_project/tps_small_data_1e-3_dec_5_eye';
    util.mkdir(out_dir_exp);
    out_file_commands=os.path.join(out_dir_exp,'commands_all.sh');
    out_dir_breakdowns=os.path.join(dir_neighbors,'small_datasets');
    file_pre='matches';
    path_to_doc='/home/maheenrashid/Downloads/horses/torch/train_tps_var_lr.th'

    commands_all=[];
    for num_data_curr in num_data:
        for num_neighbor in num_neighbors:
            file_curr=file_pre+'_'+str(num_neighbor)+'_'+str(num_data_curr);
            out_file_pre=os.path.join(out_dir_breakdowns,file_curr);
            horse_data_path=out_file_pre+post_tags[0];
            human_data_path=out_file_pre+post_tags[2];
            out_dir_curr=os.path.join(out_dir_exp,file_curr);
            command_curr=['th',path_to_doc];
            command_curr.extend(['-outDir',out_dir_curr]);
            command_curr.extend(['-horse_data_path',horse_data_path]);
            command_curr.extend(['-human_data_path',human_data_path]);
            command_curr=' '.join(command_curr);
            commands_all.append(command_curr);
    commands_all=commands_all[::-1]
    util.writeFile(out_file_commands,commands_all);
    print len(commands_all);
    print out_file_commands
    
tps_script();

In [ ]:
def main_3():
    old_horse_file='/home/SSD3/maheen-data/horse_project/data_check/horse/matches_5_train_allKP_minLoss_clean.txt';
    old_human_file='/home/SSD3/maheen-data/horse_project/data_check/aflw/matches_5_train_allKP_minLoss_clean.txt'
    old_human_file_noIm='/home/SSD3/maheen-data/horse_project/data_check/aflw/matches_5_train_allKP_minLoss_noIm_clean.txt';
    
    num_neighbors=range(5,25,5);
    num_data=range(500,3500,500);
    num_data=num_data+[3531]
    post_tags=['_horse.txt','_face.txt','_face_noIm.txt'];
    
    dir_neighbors='/home/SSD3/maheen-data/horse_project/neighbor_data';
    out_dir_breakdowns=os.path.join(dir_neighbors,'small_datasets');
    file_pre='matches';
    minloss_post='_minloss.txt';
    commands_all=[];
    files_human=[];

    for num_neighbor in num_neighbors:
        for num_data_curr in num_data:
            file_curr=file_pre+'_'+str(num_neighbor)+'_'+str(num_data_curr);
            out_file_pre=os.path.join(out_dir_breakdowns,file_curr);
            out_file_curr=out_file_pre+post_tags[1];
            files_human.append(out_file_curr);
    
    path_to_th='/home/maheenrashid/Downloads/horses/torch/train_face_system.th';
    out_dir_meta=os.path.join(dir_neighbors,'min_loss_calculation');
    util.mkdir(out_dir_meta);
    out_file_sh=os.path.join(out_dir_meta,'commands')
    commands_all=[];
    for file_curr in files_human:
        out_dir=os.path.join(out_dir_meta,file_curr[:file_curr.rindex('.')]);
        command_curr=getCommandFaceTest(path_to_th,out_dir,file_curr);
        commands_all.append(command_curr);
#         print command_curr;
    num_files=2;
    range_data=range(0,len(commands_all),len(commands_all)/num_files);
    
    if range_data[-1]!=len(commands_all):
        range_data.append(len(commands_all));
        
    for num_file in range(num_files):
        out_file_sh_curr=out_file_sh+'_'+str(num_file)+'.sh';
        data_curr=commands_all[range_data[num_file]:range_data[num_file+1]];
        
        util.writeFile(out_file_sh_curr,data_curr);
        print out_file_sh_curr;

In [ ]:
def bl_script():
    num_neighbors=range(5,10,5);
    num_data=range(500,3500,500);
    num_data=num_data+[3531]
    print num_data
    post_tags=['_horse.txt','_face.txt','_face_noIm.txt'];
    
    dir_neighbors='/home/SSD3/maheen-data/horse_project/neighbor_data';
    out_dir_breakdowns=os.path.join(dir_neighbors,'small_datasets');
    file_pre='matches';
    minloss_post='_minloss.txt';
    num_files=2;
    
    out_file_pres=[];
    for num_neighbor in num_neighbors:
        for num_data_curr in num_data:
            file_curr=file_pre+'_'+str(num_neighbor)+'_'+str(num_data_curr);
            out_file_pre=os.path.join(out_dir_breakdowns,file_curr);
            out_file_pres.append(out_file_pre);
    
    train_data_paths=[file_curr+post_tags[0][:post_tags[0].rindex('.')]+minloss_post for file_curr in out_file_pres];
    out_dir_meta='/home/SSD3/maheen-data/horse_project/face_baselines_small_data';
    util.mkdir(out_dir_meta);
    
    commands_all=[];
    
    path_to_th='/home/maheenrashid/Downloads/horses/torch/train_face_system.th';
    out_file_sh=os.path.join(out_dir_meta,'commands')
    
    commands_all=[];
    for file_curr in train_data_paths:
        file_only=os.path.split(file_curr)[1];
        out_dir=os.path.join(out_dir_meta,file_only[:file_only.rindex('.')]);
#         print file_curr,len(util.readLinesFromFile(file_curr))
        command_curr=getCommandFaceTrain(path_to_th,out_dir,file_curr);
        print command_curr
        commands_all.append(command_curr);

    range_data=range(0,len(commands_all),len(commands_all)/num_files);

    if range_data[-1]!=len(commands_all):
        range_data[-1]=(len(commands_all));
    print range_data;
    for num_file in range(num_files):
        out_file_sh_curr=out_file_sh+'_'+str(num_file)+'.sh';
        data_curr=commands_all[range_data[num_file]:range_data[num_file+1]];
        
        util.writeFile(out_file_sh_curr,data_curr);
        print out_file_sh_curr,len(data_curr);

bl_script();

In [ ]:
def full_system_script():
    num_neighbors=range(5,10,5);
#     num_data=range(500,3500,500);
#     num_data=num_data+[3531]
    num_data=[500,3531,1000,3000,1500,2500,2000];
    print num_data
    post_tags=['_horse.txt','_face.txt','_face_noIm.txt'];
    
    dir_neighbors='/home/SSD3/maheen-data/horse_project/neighbor_data';
    out_dir_breakdowns=os.path.join(dir_neighbors,'small_datasets');
    file_pre='matches';
    minloss_post='_minloss.txt';
    num_files=2;
    
    out_file_pres=[];
    for num_neighbor in num_neighbors:
        for num_data_curr in num_data:
            file_curr=file_pre+'_'+str(num_neighbor)+'_'+str(num_data_curr);
            out_file_pre=os.path.join(out_dir_breakdowns,file_curr);
            out_file_pres.append(out_file_pre);
    
    horse_train_data_paths=[file_curr+post_tags[0][:post_tags[0].rindex('.')]+minloss_post\
                            for file_curr in out_file_pres];
    human_train_data_paths=[file_curr+post_tags[2][:post_tags[2].rindex('.')]+minloss_post\
                            for file_curr in out_file_pres];
    dirs_tps=[os.path.split(file_curr)[1] for file_curr in out_file_pres];
    
    out_dir_meta='/home/SSD3/maheen-data/horse_project/full_system_small_data_eye_1e-2_10_100';
    out_dir_tps='/home/SSD3/maheen-data/horse_project/tps_small_data_1e-3_dec_5_eye';
    util.mkdir(out_dir_meta);
    
    commands_all=[];
    
    path_to_th='/home/maheenrashid/Downloads/horses/torch/train_full_system_2loss.th';
    out_file_sh=os.path.join(out_dir_meta,'commands_fixed')
    
    commands_all=[];
    for dir_tps,file_curr,file_human in zip(dirs_tps,horse_train_data_paths,human_train_data_paths):
        file_only=os.path.split(file_curr)[1];
        out_dir=os.path.join(out_dir_meta,file_only[:file_only.rindex('.')]);
#         print file_curr,len(util.readLinesFromFile(file_curr))
#         tps_model_path=os.path.join(tps_dir,)
        tps_model_path=os.path.join(out_dir_tps,dir_tps,'final/model_all_final.dat');
#         print dir_tps
#         print tps_model_path
        command_curr=getCommandFull2LossTrain(path_to_th,out_dir,file_curr,file_human,tps_model_path);
        
        print os.path.exists(tps_model_path);
        assert os.path.exists(file_curr);
        assert os.path.exists(file_human);
#         print command_curr
        commands_all.append(command_curr);

    range_data=range(0,len(commands_all),len(commands_all)/num_files);

    if range_data[-1]!=len(commands_all):
        range_data[-1]=len(commands_all);
        
    print range_data;
    
    for num_file in range(num_files):
        out_file_sh_curr=out_file_sh+'_'+str(num_file)+'.sh';
        data_curr=commands_all[range_data[num_file]:range_data[num_file+1]];
        
        util.writeFile(out_file_sh_curr,data_curr);
        print out_file_sh_curr,len(data_curr);

full_system_script();


In [ ]:
def makeComparativeGraph():
    num_neighbors=range(5,10,5);
    num_data=range(500,3500,500);
    num_data=num_data+[3531]
#     print num_data
    post_tags='_horse_minloss';
    
#     dir_neighbors='/home/SSD3/maheen-data/horse_project/neighbor_data';
#     out_dir_breakdowns=os.path.join(dir_neighbors,'small_datasets');
    file_pre='matches';
#     minloss_post='_minloss.txt';
#     num_files=2;
    out_dir_meta=['/home/SSD3/maheen-data/horse_project/full_system_small_data',\
                  '/home/SSD3/maheen-data/horse_project/face_baselines_small_data'];
    
    out_file_pres=[];
    for num_neighbor in num_neighbors:
        for num_data_curr in num_data:
            file_curr=file_pre+'_'+str(num_neighbor)+'_'+str(num_data_curr);
#             out_file_pre=os.path.join(out_dir_breakdowns,file_curr);
            out_file_pres.append(file_curr);
    log_files=[os.path.join(file_curr+post_tags,'test_images','log_test.txt') for file_curr in out_file_pres];
    losses_all=[];
    for dir_curr in out_dir_meta:
        losses=[];
        for file_curr in log_files:
            file_curr=os.path.join(dir_curr,file_curr);
            lines=util.readLinesFromFile(file_curr);
            line_last=lines[-1];
            line_last=line_last.split(' ')[-1].strip('"');
            line_last=float(line_last);
            losses.append(line_last);
        losses_all.append(losses);
    
    out_file=os.path.join(out_dir_meta[0],'comparison.png');
    
    xAndYs=[(num_data,losses_all[0]),(num_data,losses_all[1])]
    legend_entries=['Ours','Baseline']
    visualize.plotSimple(xAndYs,out_file,title='',xlabel='Training Data',\
                         ylabel='Average Euclidean Distance',legend_entries=legend_entries);
    print out_file.replace(dir_server,click_str)

#         file_curr=
#     print log_files;
#     horse_train_data_paths=[file_curr+post_tags[0][:post_tags[0].rindex('.')]+minloss_post\
#                             for file_curr in out_file_pres];
#     human_train_data_paths=[file_curr+post_tags[2][:post_tags[2].rindex('.')]+minloss_post\
#                             for file_curr in out_file_pres];
    

#     util.mkdir(out_dir_meta);
    

makeComparativeGraph();

In [ ]:
def scripts_for_min():
    increments=range(1680,10000,1680);
    print increments;
#     out_dir_meta='/home/SSD3/maheen-data/horse_project/full_system_small_data'
#     path_to_th='/home/maheenrashid/Downloads/horses/torch/train_full_system_2loss.th';
#     model_command='-full_model_path';

    out_dir_meta='/home/SSD3/maheen-data/horse_project/face_baselines_small_data'
    path_to_th='/home/maheenrashid/Downloads/horses/torch/train_face_system.th';
    model_command='-model';

    out_dir_command='-outDirTest';
    out_file_sh=os.path.join(out_dir_meta,'command_test_mid.sh');
    
    file_pres,_=getFilePres();
    
    dirs=[os.path.join(out_dir_meta,dir_curr) for dir_curr in file_pres];
    commands_all=[];
    for dir_meta in dirs:
        model_paths=[os.path.join('intermediate','model_all_'+str(num)+'.dat') for num in increments];

        model_paths=[os.path.join(dir_meta,model_path_curr) for model_path_curr in model_paths];
        out_dirs=[os.path.join(dir_meta,'test_images_'+str(num)) for num in increments]
        for model_path,out_dir in zip(model_paths,out_dirs):
            assert os.path.exists(model_path)
            command_curr=['th',path_to_th,model_command,model_path,out_dir_command,out_dir]
            command_curr=' '.join(command_curr);
            commands_all.append(command_curr);
            
    print len(commands_all);
    print commands_all[0]
    print out_file_sh
    util.writeFile(out_file_sh,commands_all);
    
    
scripts_for_min();

In [ ]:


def script_getMinLoss():
    dir_metas=['/home/SSD3/maheen-data/horse_project/full_system_small_data',
               '/home/SSD3/maheen-data/horse_project/face_baselines_small_data'];
    file_pres,num_data=getFilePres();
    loss_dir_pre='test_images_';
    loss_dir_posts=[str(num_curr) for num_curr in range(1680,9000,1680)];
    log_file='log_test.txt';
#     num_data=
    losses_all=[];
    losses_all_end=[];
    min_loss_iter_all=[];
    for dir_meta in dir_metas:
        loss_curr=[];
        loss_end=[];
        min_loss_iter=[];
        for file_pre in file_pres:
            min_loss,min_loss_post=getMinLoss(os.path.join(dir_meta,file_pre),loss_dir_pre,loss_dir_posts,log_file);
#             print loss_dir_posts
            loss_end_curr,min_loss_post_end=getMinLoss(os.path.join(dir_meta,file_pre),loss_dir_pre,['8400'],log_file);
            loss_curr.append(min_loss);
            loss_end.append(loss_end_curr);
            min_loss_iter.append(min_loss_post);
            print min_loss,min_loss_post,loss_end_curr,min_loss_post_end;

        losses_all.append(loss_curr);
        losses_all_end.append(loss_end)
        min_loss_iter_all.append(min_loss_iter);
        
    out_file=os.path.join(dir_metas[0],'comparison_best.png');
    print len(file_pres),len(losses_all[0]),len(losses_all[1]),len(num_data)
    xAndYs=[(num_data,losses_all[0]),(num_data,losses_all[1]),\
           (num_data,losses_all_end[0]),(num_data,losses_all_end[1])];
    legend_entries=['Ours Best','Baseline Best','Ours 8400','Baseline 8400'];
    visualize.plotSimple(xAndYs,out_file,title='',xlabel='Training Data',\
                         ylabel='Average Euclidean Distance',legend_entries=legend_entries);
    print out_file.replace(dir_server,click_str)
    
    
print 'hello'
script_getMinLoss();
print 'hello post'

In [ ]:
print float('inf')

In [ ]:
dir_curr='/home/SSD3/maheen-data/temp/checkDualRotation';
visualize.writeHTMLForFolder(dir_curr);
print os.path.join(dir_curr,os.path.split(dir_curr)[1]+'.html').replace(dir_server,click_str);

In [ ]:
path_to_th='/home/maheenrashid/Downloads/horses/torch/train_full_system_2loss.th';
tps_model_path='';
out_dir_meta='/home/SSD3/maheen-data/horse_project/full_system_2loss_search';
util.mkdir(out_dir_meta);
out_file_sh=os.path.join(out_dir_meta,'commands');
# outDir='';
learningRate=1e-2;
multiplier_pairs=[(1/100.0,1/100.0),(1/10.0,1/10.0),(1/10.0,1/100.0)];
horse_data_path='/home/SSD3/maheen-data/horse_project/neighbor_data/small_datasets/matches_5_3531_horse_minloss.txt';
human_data_path='/home/SSD3/maheen-data/horse_project/neighbor_data/small_datasets/matches_5_3531_face_noIm_minloss.txt';
tps_model_path='/home/SSD3/maheen-data/horse_project/tps_small_data_1e-3_dec_5/matches_5_3531/final/model_all_final.dat';

# th /home/maheenrashid/Downloads/horses/torch/train_full_system_2loss.th 
# -outDir /home/SSD3/maheen-data/horse_project/full_system_small_data/matches_5_3531_horse_minloss -horse_data_path  -human_data_path 

commands_all=[];
for multiplierMid,multiplierBottom in multiplier_pairs:
    out_dir_curr=os.path.join(out_dir_meta,'test_'+\
                              '_'.join([str(num) for num in [learningRate,multiplierMid,multiplierBottom]]));
    command_curr=['th',path_to_th];
    command_curr.extend(['-outDir',out_dir_curr]);
    command_curr.extend(['learningRate',str(learningRate)]);
    command_curr.extend(['multiplierBottom',str(multiplierBottom)]);
    command_curr.extend(['multiplierMid',str(multiplierMid)]);
    command_curr.extend(['-horse_data_path',horse_data_path]);
    command_curr.extend(['-human_data_path',human_data_path]);
    command_curr.extend(['-tps_model_path',tps_model_path]);
    command_curr=' '.join(command_curr);
    print command_curr;
    commands_all.append(command_curr);
    
num_files=2;
range_data=range(0,len(commands_all),len(commands_all)/num_files);

if range_data[-1]!=len(commands_all):
    range_data.append(len(commands_all));

for num_file in range(num_files):
    out_file_sh_curr=out_file_sh+'_'+str(num_file)+'.sh';
    data_curr=commands_all[range_data[num_file]:range_data[num_file+1]];

    util.writeFile(out_file_sh_curr,data_curr);
    print out_file_sh_curr;

In [ ]:
import random;
dir_meta='/home/SSD3/maheen-data/horse_project/sheep_data'
in_file='sheep.txt';
test_num=100;
out_file_train=os.path.join(dir_meta,'trainImageList.txt');
out_file_test=os.path.join(dir_meta,'testImageList.txt')
lines=util.readLinesFromFile(os.path.join(dir_meta,in_file));
train_num=len(lines)-test_num;
random.shuffle(lines);
train_split=lines[:train_num];
test_split=lines[train_num:];
util.writeFile(out_file_train,train_split);
util.writeFile(out_file_test,test_split);



In [5]:
def writeMinLossFilesSimple(new_files,minloss_post,loss_file):
#     new_files=[out_file_pre+post_tag_curr for post_tag_curr in post_tags];
    horse_data=util.readLinesFromFile(new_files[0]);
    horse_data=np.array(horse_data);
    horse_data_uni=np.unique(horse_data);
    face_data=util.readLinesFromFile(new_files[1]);
    face_data_noIm=util.readLinesFromFile(new_files[2]);
    assert len(face_data)==len(face_data_noIm);
    
    loss_all=np.load(loss_file);
    loss_all=loss_all[:len(face_data)];
    assert loss_all.shape[0]==len(face_data);
    
    new_data=[[],[],[]];
    for idx_curr,horse_curr in enumerate(horse_data_uni):
        idx_rel=np.where(horse_data==horse_curr)[0];
        loss_rel=loss_all[idx_rel];
        min_idx=np.argmin(loss_rel);
        min_idx_big=idx_rel[min_idx];
        assert loss_rel[min_idx]==loss_all[min_idx_big];
        new_data[0].append(horse_curr);
        new_data[1].append(face_data[min_idx_big]);
        new_data[2].append(face_data_noIm[min_idx_big]);
  
    new_files_out=[new_file_curr[:new_file_curr.rindex('.')]+minloss_post for new_file_curr in new_files];
    for new_file_to_write,data_to_write in zip(new_files_out,new_data):
        print new_file_to_write,len(data_to_write);
        util.writeFile(new_file_to_write,data_to_write);


def createSheepFiles():
    dir_meta='/home/SSD3/maheen-data/horse_project/data_check'
    dirs=[os.path.join(dir_meta,'sheep'),os.path.join(dir_meta,'aflw')];
    files=['matches_5_sheep_test_allKP','matches_5_sheep_train_allKP']
    post_tags=[['.txt'],['.txt','_noIm.txt']];
    loss_dir_post=['test_face','train_face'];
    loss_dir='/home/SSD3/maheen-data/horse_project/sheep_models/tps'
    minloss_post='_minloss.txt'
    for idx_file_curr,file_curr in enumerate(files):
        files_in=[];
        for idx_dir_curr,dir_curr in enumerate(dirs):
            for post_tag in post_tags[idx_dir_curr]:
                files_in.append(os.path.join(dir_curr,file_curr+post_tag));

        for file_curr in files_in:
            print file_curr,os.path.exists(file_curr);

        loss_dir_curr=os.path.join(loss_dir,loss_dir_post[idx_file_curr]);
        loss_file=os.path.join(loss_dir_curr,'loss_final_val_ind.npy');
        print loss_file,os.path.exists(loss_file)
        print 'done'
        writeMinLossFilesSimple(files_in,minloss_post,loss_file)

        createSheepFiles();

/home/SSD3/maheen-data/horse_project/data_check/sheep/matches_5_sheep_test_allKP.txt True
/home/SSD3/maheen-data/horse_project/data_check/aflw/matches_5_sheep_test_allKP.txt True
/home/SSD3/maheen-data/horse_project/data_check/aflw/matches_5_sheep_test_allKP_noIm.txt True
/home/SSD3/maheen-data/horse_project/sheep_models/tps/test_face/loss_final_val_ind.npy True
done
/home/SSD3/maheen-data/horse_project/data_check/sheep/matches_5_sheep_test_allKP_minloss.txt 99
/home/SSD3/maheen-data/horse_project/data_check/aflw/matches_5_sheep_test_allKP_minloss.txt 99
/home/SSD3/maheen-data/horse_project/data_check/aflw/matches_5_sheep_test_allKP_noIm_minloss.txt 99
/home/SSD3/maheen-data/horse_project/data_check/sheep/matches_5_sheep_train_allKP.txt True
/home/SSD3/maheen-data/horse_project/data_check/aflw/matches_5_sheep_train_allKP.txt True
/home/SSD3/maheen-data/horse_project/data_check/aflw/matches_5_sheep_train_allKP_noIm.txt True
/home/SSD3/maheen-data/horse_project/sheep_models/tps/train_fac

In [7]:
out_dir_meta='/home/SSD3/maheen-data/horse_project/sheep_models'
val_data_path=['-val_data_path',\
               '/home/SSD3/maheen-data/horse_project/data_check/sheep/matches_5_sheep_test_allKP_minloss.txt'];
horse_data_path=['-horse_data_path',\
               '/home/SSD3/maheen-data/horse_project/data_check/sheep/matches_5_sheep_train_allKP_minloss.txt'];
human_data_path=['-human_data_path',\
               '/home/SSD3/maheen-data/horse_project/data_check/aflw/matches_5_sheep_train_allKP_noIm_minloss.txt'];
tps_model_path=['-tps_model_path',os.path.join(out_dir_meta,'tps','final','model_all_final.dat')];
out_dir=['-outDir',os.path.join(out_dir_meta,'full_system_2loss')];
path_to_th='/home/maheenrashid/Downloads/horses/torch/train_full_system_2loss.th';
th=['th',path_to_th];
command=th+out_dir+tps_model_path+val_data_path+human_data_path+horse_data_path;
command=' '.join(command);
print command;




                



th /home/maheenrashid/Downloads/horses/torch/train_full_system_2loss.th -outDir /home/SSD3/maheen-data/horse_project/sheep_models/full_system_2loss -tps_model_path /home/SSD3/maheen-data/horse_project/sheep_models/tps/final/model_all_final.dat -val_data_path /home/SSD3/maheen-data/horse_project/data_check/sheep/matches_5_sheep_test_allKP_minloss.txt -human_data_path /home/SSD3/maheen-data/horse_project/data_check/aflw/matches_5_sheep_train_allKP_noIm_minloss.txt -horse_data_path /home/SSD3/maheen-data/horse_project/data_check/sheep/matches_5_sheep_train_allKP_minloss.txt
